In [1]:
import os

In [4]:
%pwd

'c:\\Users\\mdgha\\OneDrive\\Documents\\Ghayaz Personal\\MLE, Data Science\\MLOps'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
!pip install ensure


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from ChickenDiseaseClassifier.constant import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
        

In [13]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassifier.utils.common import get_size
from ChickenDiseaseClassifier import logger

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_zip(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )

            logger.info(f"{filename} downloaded with following info:\n {headers}")

        else:
            logger.info(f"File already exists: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    downloaded_zip = data_ingestion.download_zip()
    extracted_zip = data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-08-01 20:20:23,668: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-01 20:20:23,681: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-01 20:20:23,689: INFO: common: created directory at: artifacts]
[2025-08-01 20:20:23,694: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-01 20:20:52,061: INFO: 1165745557: artifacts/data_ingestion/data.zip downloaded with following info:
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E122:2AFEF0:3C314:79D5A:688CD430
Accept-Ranges: bytes
Date: Fri, 01 Aug 2025 14:50:25 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10240-MAA
X-Cache: M